# World Model V3 Diagnostics Smoke Test

Dieses Notebook führt einen kurzen Smoke-Test für `src.analysis.diagnostics.run_diagnostics_for_run` aus:

- Lädt ein bereits trainiertes World Model V3 Experiment
- Führt die Diagnostics aus (inkl. EOL-Metriken und HI-Trajektorien)
- Zeigt die Test-Metriken aus dem Training und die Diagnostics-EOL-Metriken nebeneinander an
- Rendert **ein HI+RUL-Trajektorien-Plot** aus den von `diagnostics.py` erzeugten Dateien

Passen Sie lediglich `dataset_name` und `run_name` an Ihr Experiment an.


In [ ]:
# Imports & basic config
%matplotlib inline

import json
from pathlib import Path

import matplotlib
matplotlib.use("module://matplotlib_inline.backend_inline")

import matplotlib.pyplot as plt
from IPython.display import Image, display
import torch

from src.analysis.diagnostics import run_diagnostics_for_run

# === CONFIG ===
# Passe diese beiden Variablen an deinen Experiment-Run an
dataset_name = "FD002"  # z.B. "FD001", "FD002", ...
run_name = "fd002_world_phase5_universal_v3_residual"  # Experimentordner unter results/<dataset>/

exp_dir = Path("results")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"Experiment directory: {exp_dir / dataset_name.lower() / run_name}")


In [ ]:
# Run diagnostics.py for the selected experiment

_ = run_diagnostics_for_run(
    exp_dir=exp_dir,
    dataset_name=dataset_name,
    run_name=run_name,
    device=device,
)

print("\nDiagnostics run finished.")


In [ ]:
# Compare training test_metrics with diagnostics EOL metrics

experiment_dir = exp_dir / dataset_name.lower() / run_name
summary_path = experiment_dir / "summary.json"

if not summary_path.exists():
    raise FileNotFoundError(f"summary.json not found at {summary_path}")

with open(summary_path, "r") as f:
    summary = json.load(f)

train_test_metrics = summary.get("test_metrics", {})

eol_diag = summary.get("eol_diagnostics", {})

diagnostics_metrics = {
    "rmse":  eol_diag.get("rmse"),
    "mae":   eol_diag.get("mae"),
    "bias":  eol_diag.get("bias"),
    "r2":    eol_diag.get("r2"),
    "nasa_mean": eol_diag.get("nasa_mean"),
    "nasa_sum":  eol_diag.get("nasa_sum"),
    "num_engines": eol_diag.get("num_engines"),
}

print("=== Test metrics from training (summary['test_metrics']) ===")
for k, v in train_test_metrics.items():
    print(f"{k:10s}: {v}")

print("\n=== Diagnostics EOL metrics (summary['eol_diagnostics']) ===")
for k, v in diagnostics_metrics.items():
    print(f"{k:10s}: {v}")


In [ ]:
# Show one HI trajectory plot produced by diagnostics

hi_plot_path = experiment_dir / "hi_rul_10_degraded.png"

if not hi_plot_path.exists():
    # Fallback: try the simpler HI-only plot
    hi_plot_path = experiment_dir / "hi_trajectories_10_engines.png"

print(f"Loading HI plot from: {hi_plot_path}")

if not hi_plot_path.exists():
    raise FileNotFoundError(f"HI trajectory plot not found in {experiment_dir}")

display(Image(filename=str(hi_plot_path)))
